## Векторная модель текста в NLTK
### Задание 1
Возьмем файл alice.txt с прошлого семинара. Мы уже научились разбивать его на токены и удалять стоп-слова.
Используя nltk.FreqDist, выведите 20 самых часто встречающихся слов в тексте (не удаляя стоп символы).

Ответьте на вопросы:

    Какое слово стоит на 1 месте?
    Сколько раз встречается слово, стоящее на 20 месте?
    Сколько не стоп-слов среди первых десяти?
    Сколько раз встречается слово "крокет"? (понятно, что оно не входит в top20)


In [ ]:
# d1 = nltk.FreqDist(l1)
# d1.most_common(10)

# d3['...'], d3.freq('...')

### Задание 2
Избавьтесь от стоп-слов с помощью списка stopwords из nltk.corpus. Используйте pymorphy2 или Mystem, чтобы привести слова к нормальной форме. Ответьте на вопросы:

    Какое слово стоит на 1 месте?
    Сколько раз встречается слово, стоящее на 20 месте?
    Остались ли какие-то стоп-слова среди первых 10? 20?
    Сколько раз встречается слово "крокет"? Почему результат изменился?


In [ ]:
# morph = pymorphy2.MorphAnalyzer()
# l3 = [morph.parse(token)[0].normal_form for token in l1 if not token in stopwords.words('russian')]

# d3['...'], d3.freq('...')

Давайте пострим кумулятивный график частот слов. Как он выглядит и почему так? Попробуйте поменять количество слов, какие тенденции вы видите?


In [ ]:
d3.plot (50, cumulative = True)

### Задание 3

Найдите распределение длин слов в тексте, выведите top5 и ответьте на вопросы:

    Сколько всего в данном тексте вариантов длин слов (вместе со стоп-словами)?
    Слов какой длины в тексте больше всего?
    Какую долю занимают слова этой длины?
    Сколько слов длины 1?

Выведите в алфавитном поредке длинные слова (длиннее 15 символов).

    Сколько их?

In [ ]:
# fdist = nltk.FreqDist(len(w) for w in l1)  
# print(fdist)  
# print (fdist.most_common(5))
# print (fdist.freq(fdist.max()))

# long_words = [w for w in l1 if len(w) > 10]
# sorted(long_words)

### Задание 4

Превратите строку в NLTK-текст и проверьте, как работают спецфункции, на собственных примерах.


In [ ]:
# text = nltk.Text(l1)
# print (text.concordance(".....")) 
# print (text.similar('......'))
# print (text.common_contexts([".....", "......"]))
# print (text.collocations(30))

## Эмбеддинги


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
from sklearn.metrics import *
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')
random.seed(1228)

%matplotlib inline

Скачиваем куски датасета твитов ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/ilkte35m35l38mr/negative.sql).

In [ ]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

In [ ]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

Загружаем лемматизированные статьи без стоп-слов и создаем массив текстов

In [ ]:
from pymystem3 import Mystem
import re


m = Mystem()


regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""



def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [ ]:
df_neg = pd.read_csv("negative.csv?dl=0", sep=';', header = None, usecols = [3])
df_pos = pd.read_csv("positive.csv?dl=0", sep=';', header = None, usecols = [3])
df_neg['sent'] = 'neg'
df_pos['sent'] = 'pos'
df_pos['text'] = df_pos[3]
df_neg['text'] = df_neg[3]
df = pd.concat([df_neg, df_pos])
df = df[['text', 'sent']]
%time df.text = df.text.apply(words_only)
%time df.text = df.text.apply(lemmatize)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.tail()

In [ ]:
texts = [df.text.iloc[i].split() for i in range(len(df))]

### Обучение модели в gensim

In [ ]:
texts[0]

In [ ]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(texts, size=100, window=5, min_count=5, workers=4)
model.save("word2v.model")

In [ ]:
# здесь можно посмотреть все слова, к которым мы имеем вектора
# model.vocab

Загружаем обученную модель (для скорости):

In [ ]:
# from gensim.models import Word2Vec
# model = Word2Vec.load("sent_w2v.model")

Теперь можно воспользjваться функциями поиска похожих / непохожих слов на данное

    model.most_similar
    model.doesnt_match
    etc

In [ ]:
model.most_similar("корпоратив")

In [ ]:
model.most_similar(positive=["король","добрый"], negative=["хороший"])

In [ ]:
model.doesnt_match("борщ сметана макароны пирожок котлета".split())

### Задание 5
Выделите top 100 самых частотных слов и получите для них только что обученные вектора

In [ ]:
from nltk import FreqDist

fd = FreqDist()
# your code is hear

top_words = # your code is hear

In [ ]:
model['рабочий']

In [ ]:
top_words_vec = # your code is hear

Визуализируем вектора. Для этого воспользуемся методом снижения размерности t-sne

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Теперь нарисуем облако слов

In [ ]:
from wordcloud import *
word_freq = [i for i in fd.most_common(100)]
wd = WordCloud(background_color = 'white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure()
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()

### Кластеризация слов

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(top_words_vec)

In [ ]:
from scipy.cluster.hierarchy import  ward, dendrogram

linkage_matrix = ward(dist) 

fig, ax = plt.subplots(figsize=(10, 100)) 
ax = dendrogram(linkage_matrix, orientation="right", labels=top_words);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() 

plt.savefig('w2v_cluster.png', dpi=200) #save figure as ward_clusters

## Языковые модели

### Модели биграм в NLTK

In [ ]:
import nltk

In [ ]:
names = [name.strip().lower() for name in open('dinos.txt').readlines()]
print(names[:10])

In [ ]:
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)

print(list(freq.keys()))

In [ ]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))
print(cfreq['a']

In [ ]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

In [ ]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l
print('p(a) = %1.4f' %unigram_prob('a'))

In [ ]:
# можно порождать случайные символы с учётом предыдущих 
cprob['a'].generate()

### Задание 6

Напишите функцию для генерации нового имени динозавра фиксированной длины.